## Imports 

In [9]:
# TODO 
# the dates might mess up if it's a single digit day of 
# the month since I just take the first 5 characters of the date string at fivethirtyeight

In [10]:
import pandas as pd 
import requests
from bs4 import BeautifulSoup 
import pandas as pd 
import re
import time    

## Helper Function 

In [11]:
# Convert money lines to probabilites 
def to_prob(money_line):
    if money_line < 0: 
        prob = -money_line / (-money_line + 100) 
    elif money_line > 0: 
        prob = 100 / (money_line + 100) 
    else: 
        prob = None 

    return (prob) 

## Scrape 538

In [12]:
url = 'https://projects.fivethirtyeight.com/2019-mlb-predictions/games/'
page_response = requests.get(url)
page_content = BeautifulSoup(page_response.text, 'html.parser')

In [13]:
rows = page_content.findAll(class_="tr")

dict_list = []

for row in rows:
    try:
        date = row.find(class_='td td-datetime').getText()
    except:
        score = None
    long_name = row.find(class_='team-name long').getText()
    short_name = row.find(class_='team-name short').getText()
    pitcher = row.find(class_='pitcher-name').getText()
    num_rating = row.find(class_='td number td-number rating-adj').getText().replace('=', '')
    win_prob = row.find(class_='td number td-number win-prob').getText()
    
    
    try:
        score = row.find(class_='td number td-number score').getText()
    except:
        score = None
    
    d = {
        'date': date[:4],
        'long_name': long_name,
        'short_name': short_name,
        'pitcher': pitcher,
        'num_rating': num_rating, 
        'win_prob': win_prob.replace('%', ''),
        'score': score
    }
    
    dict_list.append(d)
    
fte_df = pd.DataFrame(dict_list)
fte_df

,date,long_name,num_rating,pitcher,score,short_name,win_prob
0,7/20,Rockies,1485,Senzatela,,COL,31
1,7/20,Yankees,1622,Tanaka,,NYY,69
2,7/20,Padres,1488,Lucchesi,,SD,40
3,7/20,Cubs,1556,Quintana,,CHC,60
4,7/20,Mets,1458,Lockett,,NYM,42
5,7/20,Giants,1517,Samardzija,,SF,58
6,7/20,Blue Jays,1468,Thornton,,TOR,55
7,7/20,Tigers,1433,Norris,,DET,45
8,7/20,White Sox,1489,Giolito,,CHW,43
9,7/20,Rays,1541,Yarbrough,,TB,57


## Scrape ESPN

In [21]:
# Get Page and Content 
url = 'http://www.espn.com/mlb/lines'
page_response = requests.get(url)
page_content = BeautifulSoup(page_response.text, 'html.parser')

# Pull Rows of Data
rows = page_content.findAll(class_="oddrow")
rows = rows + (page_content.findAll(class_="evenrow"))

In [29]:
# List Precursor 
dict_list = []

# Main Text Parsing of MLB Games 
for row in rows: 
    if row.findAll("td")[0].getText() == 'Westgate':
        bookee = (row.findAll("td")[0].getText())

        row = (str(row.findAll("td")[1]).replace("<td style=\"text-align:center;\">", "").replace("</td>","").split("<br/>"))
        
        if (row[0] == 'N/A'):
            pass
        else:
            team_1 = row[0].split(":")[0]
            team_2 = row[1].split(":")[0]
            moneyline_1 = row[0].split(":")[1].strip()
            moneyline_2 = row[1].split(":")[1].strip()    
        
        #print(bookee, team_1, moneyline_1, team_2, moneyline_2)
        
        d1 = {
        'bookee': bookee,
        'short_name': team_1,
        'moneyline': moneyline_1
        }
        
        d2 = {
        'bookee': bookee,
        'short_name': team_2,
        'moneyline': moneyline_2    
        }
         
        dict_list.append(d1)
        dict_list.append(d2)
        
        
# Create df and Features 
mlb_df = pd.DataFrame(dict_list)   
mlb_df['prob'] = [round(to_prob(int(i))*100) for i in mlb_df['moneyline']]
mlb_df['date'] = str(time.strftime("%-m/%d")) #removes leading 0   
mlb_df

,bookee,moneyline,short_name,prob,date
0,Westgate,220,COL,31,7/20
1,Westgate,-250,NYY,71,7/20
2,Westgate,220,COL,31,7/20
3,Westgate,-250,NYY,71,7/20
4,Westgate,110,NYM,48,7/20
5,Westgate,-120,SF,55,7/20
6,Westgate,110,NYM,48,7/20
7,Westgate,-120,SF,55,7/20
8,Westgate,-120,TOR,55,7/20
9,Westgate,110,DET,48,7/20


## Compare Probabilities

In [30]:
result = pd.merge(fte_df,
                 mlb_df,
                 on=['date', 'short_name'])

result['difference'] = result['win_prob'].astype('int64') - result['prob'].astype('int64')
result

result.sort_values(by='difference', ascending=False)

,date,long_name,num_rating,pitcher,score,short_name,win_prob,bookee,moneyline,prob,difference
6,7/20,Giants,1517,Samardzija,,SF,58,Westgate,-120,55,3
7,7/20,Giants,1517,Samardzija,,SF,58,Westgate,-120,55,3
25,7/20,Diamondbacks,1580,Greinke,,ARI,59,Westgate,-130,57,2
19,7/20,Athletics,1537,Anderson,,OAK,45,Westgate,130,43,2
18,7/20,Athletics,1537,Anderson,,OAK,45,Westgate,130,43,2
14,7/20,Cardinals,1533,Mikolas,,STL,47,Westgate,120,45,2
11,7/20,Orioles,1435,Eshelman,,BAL,37,Westgate,175,36,1
16,7/20,Royals,1454,Junis,,KC,40,Westgate,155,39,1
1,7/20,Rockies,1485,Senzatela,,COL,31,Westgate,220,31,0
13,7/20,Pirates,1528,Musgrove,,PIT,53,Westgate,-115,53,0
